# EVENT ANALYSIS

In [2]:
# Importing Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Setting the path to access the data
path = r"C:\Users\Mihir\8_CAPSTONE PROJECT\S1_Data_Preprocessing\1_Preprocessing_Data_&_EDA_LEVEL_0\\"
path

'C:\\Users\\Mihir\\8_CAPSTONE PROJECT\\S1_Data_Preprocessing\\1_Preprocessing_Data_&_EDA_LEVEL_0\\\\'

In [4]:
# reading required csv files
appearances = pd.read_csv(path + "appearances_df_no_win.csv")
players = pd.read_csv(path + "players_df_winsorised.csv")
games = pd.read_csv(path + "games_df_winsorised.csv")
game_events = pd.read_csv(path + "game_events_df_no_win.csv")
test = pd.read_csv(path + "test data.csv")

# HYPOTHESIS TESTING
## Goals Scored in the Last 15 Minutes of a Game Are More Likely to Impact the Final Result (Win/Loss)

In [6]:
# Getting Data for Goals scored after 75 minutes in Game
last_15_min_goals = game_events[(game_events['type'] == 'Goals') & (game_events['minute'] > 75)]

# Creating Results column for Wins and Losses and Ties
games['result'] = np.where(games['home_club_goals'] > games['away_club_goals'], 1, 
                           np.where(games['home_club_goals'] < games['away_club_goals'], 0, -1))  # 1 = Win, 0 = Loss, -1 = Tie

# Merge the filtered goals data with games data on game_id
goals_with_results = last_15_min_goals.merge(games, how='inner', on='game_id')

# Add a column to identify games with late goals and without late goals
games['has_late_goal'] = np.where(games['game_id'].isin(goals_with_results['game_id']), 1, 0)

# Separate into two groups: games with late goals and games without late goals
late_goals_group = games[games['has_late_goal'] == 1]['result']
no_late_goals_group = games[games['has_late_goal'] == 0]['result']

In [8]:
## USING Z TEST
# Calculate means and standard deviations of both groups
mean_late_goals = late_goals_group.mean()
mean_no_late_goals = no_late_goals_group.mean()

std_late_goals = late_goals_group.std()
std_no_late_goals = no_late_goals_group.std()

n1 = len(late_goals_group)
n2 = len(no_late_goals_group)

# Calculate the Z-statistic
from scipy import stats
z_stat = (mean_late_goals - mean_no_late_goals) / np.sqrt((std_late_goals**2 / n1) + (std_no_late_goals**2 / n2))

# Find the Z-critical value for a 95% confidence level (two-tailed)
z_critical = stats.norm.ppf(1 - 0.05 / 2)

# Calculate the p-value
p_value = 2 * (1 - stats.norm.cdf(abs(z_stat)))  # Two-tailed test

# 8. Print results
print(f"Z-statistic: {z_stat}")
print(f"Z-critical value (95% confidence, two-tailed): {z_critical}")
print(f"P-value: {p_value}")
print()
# Interpretation of results
if p_value < 0.05:# abs(z_stat) > z_critical:
    print("Reject the null hypothesis: Goals scored in the last 15 minutes significantly impact the match outcome.")
else:
    print("Fail to reject the null hypothesis: Goals scored in the last 15 minutes do not significantly impact the match outcome.")

Z-statistic: 1.3154554048685592
Z-critical value (95% confidence, two-tailed): 1.959963984540054
P-value: 0.18835689549586832

Fail to reject the null hypothesis: Goals scored in the last 15 minutes do not significantly impact the match outcome.


In [10]:
## USING T-TEST
# Calculate degrees of freedom
n1 = len(late_goals_group)
n2 = len(no_late_goals_group)
df = n1 + n2 - 2

# Find the t-critical value for a two-tailed test with 95% confidence level (alpha = 0.05)
from scipy import stats
alpha = 0.05
t_critical = stats.t.ppf(1 - alpha/2, df)

# Conduct an independent t-test
t_stat, p_value = stats.ttest_ind(late_goals_group, no_late_goals_group)


# Print results
print(f"T-statistic: {t_stat}")
print(f"T-Critical: {t_critical}")
print(f"P-value: {p_value}")
print()
# Interpretation of results
if p_value < 0.05:
    print("Reject the null hypothesis: Goals scored in the last 15 minutes significantly impact the match outcome.")
else:
    print("Fail to reject the null hypothesis: Goals scored in the last 15 minutes do not significantly impact the match outcome.")

T-statistic: 1.318336281688606
T-Critical: 1.9606906029345763
P-value: 0.18748354618344487

Fail to reject the null hypothesis: Goals scored in the last 15 minutes do not significantly impact the match outcome.
